In [1]:
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg19 import preprocess_input
from keras.models import Model, Sequential
from keras.layers import Input,Flatten, Dense, Dropout, Convolution2D, MaxPooling2D,GlobalAveragePooling2D
import numpy as np
import keras
import tensorflow as tf
from keras.datasets import cifar10
from keras import backend as K
from sklearn.preprocessing import OneHotEncoder

### 使用 ResNet50 进行 ImageNet 分类

In [7]:
model = ResNet50(weights='imagenet')
img_path = 'data\Tano.JPG'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)
# 将结果解码为元组列表 (class, description, probability)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n02106166', 'Border_collie', 0.891881), ('n02106030', 'collie', 0.059547905), ('n02364673', 'guinea_pig', 0.018965976)]


### 使用VGG16提取特征

In [15]:
model = VGG16(weights='imagenet', include_top=False)
img_path = 'data\Tano.JPG'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
features = model.predict(x)
features.shape

(1, 7, 7, 512)

### 从VGG19 的任意中间层中抽取特征

In [20]:
base_model = VGG19(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)

img_path = 'data\Tano.JPG'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
block4_pool_features = model.predict(x)
block4_pool_features.shape

(1, 14, 14, 512)

### 在新类上微调 InceptionV3

In [24]:
# 构建不带分类器的预训练模型
base_model = InceptionV3(weights='imagenet', include_top=False)
# 添加全局平均池化层
x = base_model.output
x = GlobalAveragePooling2D()(x)
# 添加一个全连接层
x = Dense(1024, activation='relu')(x)
# 添加一个分类器，假设我们有200个类
predictions = Dense(200, activation='softmax')(x)
# 构建我们需要训练的完整模型
model = Model(inputs=base_model.input, outputs=predictions)
# 首先，我们只训练顶部的几层（随机初始化的层）
# 锁住所有 InceptionV3 的卷积层
for layer in base_model.layers:
    layer.trainable = False
# 编译模型（一定要在锁层以后操作）
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
# 在新的数据集上训练几代
# model.fit_generator(...)
# 现在顶层应该训练好了，让我们开始微调 Inception V3 的卷积层。
# 我们会锁住底下的几层，然后训练其余的顶层。
# 让我们看看每一层的名字和层号，看看我们应该锁多少层呢：
# for i, layer in enumerate(base_model.layers):
#    print(i, layer.name)
# 我们选择训练最上面的两个 Inception block
# 也就是说锁住前面249层，然后放开之后的层。
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True
# 我们需要重新编译模型，才能使上面的修改生效
# 让我们设置一个很低的学习率，使用 SGD 来微调
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')
# 我们继续训练模型，这次我们训练最后两个 Inception block
# 和两个全连接层
# model.fit_generator(...)

### 通过自定义输入张量构建 InceptionV3

In [27]:
# 这也可能是不同的 Keras 模型或层的输出
input_tensor = Input(shape=(224, 224, 3))  # 假定 K.image_data_format() == 'channels_last'
model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=True)

### 使用ResNet 做 Trnasfer Learning

In [8]:
import time
startT=time.time()
model=ResNet50(input_shape=(32,32,3), include_top=False, weights='imagenet',pooling=None, classes=10)
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(units=128, activation='relu')(x)
x=Dropout(rate=0.1)(x)
predictions = Dense(units=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape) #(50000, 32, 32, 3)
## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=1)
print(round(time.time()-startT,3),' second')

Model深度： 179
(50000, 32, 32, 3)
1563/1563 [==============================] - 1298s 830ms/step - loss: 1.7528 - accuracy: 0.3791
1306.064  second


### [更多模型](https://keras.io/zh/applications/)

In [7]:
Xception(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
VGG19(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
InceptionV3(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
keras.applications.resnet.ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
keras.applications.resnet.ResNet101(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
keras.applications.resnet.ResNet152(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
keras.applications.resnet_v2.ResNet50V2(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
keras.applications.resnet_v2.ResNet101V2(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
keras.applications.resnet_v2.ResNet152V2(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
keras.applications.inception_resnet_v2.InceptionResNetV2(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
keras.applications.densenet.DenseNet121(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
keras.applications.densenet.DenseNet169(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
keras.applications.densenet.DenseNet201(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
keras.applications.nasnet.NASNetLarge(input_shape=None, include_top=True, weights='imagenet', input_tensor=None, pooling=None, classes=1000)
keras.applications.nasnet.NASNetMobile(input_shape=None, include_top=True, weights='imagenet', input_tensor=None, pooling=None, classes=1000)
keras.applications.mobilenet_v2.MobileNetV2(input_shape=None, alpha=1.0, include_top=True, weights='imagenet', input_tensor=None, pooling=None, classes=1000)

![ResNet_Structure](data\CNN_models.jpg)

### [常用数据集](https://keras.io/zh/applications/)

In [8]:
# 训练集为 60,000 张 28x28 像素灰度图像，测试集为 10,000 同规格图像，总共 10 类数字标签。
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# 训练集为 60,000 张 28x28 像素灰度图像，测试集为 10,000 同规格图像，总共 10 类时尚物品标签。
from keras.datasets import fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
# 50,000 张 32x32 彩色训练图像数据，以及 10,000 张测试图像数据，总共分为 10 个类别
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# 50,000 张 32x32 彩色训练图像数据，以及 10,000 张测试图像数据，总共分为 100 个类别。
from keras.datasets import cifar100


(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')
# # 数据集来自 IMDB 的 25,000 条电影评论，以情绪（正面/负面）标记。
# # 评论已经过预处理，并编码为词索引（整数）的序列表示。
# 为了方便起见，将词按数据集中出现的频率进行索引，例如整数 3 编码数据中第三个最频繁的词。
# 这允许快速筛选操作，例如：「只考虑前 10,000 个最常用的词，但排除前 20 个最常见的词」。
from keras.datasets import imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(path="imdb.npz",
                                                      num_words=None,
                                                      skip_top=0,
                                                      maxlen=None,
                                                      seed=113,
                                                      start_char=1,
                                                      oov_char=2,
                                                      index_from=3)
# 数据集来源于路透社的 11,228 条新闻文本，总共分为 46 个主题。
# 与 IMDB 数据集一样，每条新闻都被编码为一个词索引的序列（相同的约定）
from keras.datasets import reuters
(x_train, y_train), (x_test, y_test) = reuters.load_data(path="reuters.npz",
                                                         num_words=None,
                                                         skip_top=0,
                                                         maxlen=None,
                                                         test_split=0.2,
                                                         seed=113,
                                                         start_char=1,
                                                         oov_char=2,
                                                         index_from=3)
# 样本包含 1970 年代的在波士顿郊区不同位置的房屋信息，总共有 13 种房屋属性。 目标值是一个位置的房屋的中值（单位：k$）
from keras.datasets import boston_housing
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()